# Lets Start NLP Module

In [18]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import re

# print(data.columns)
# Extract 'problems' and 'findings' columns
df= pd.read_csv("indiana_reports.csv")[["Problems","findings"]]
#print df train element by element
print(df.iloc[0])




Problems                                               normal
findings    The cardiac silhouette and mediastinum size ar...
Name: 0, dtype: object


In [19]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)
#Also print gpu name
if device == 'cuda':
    print(cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 3060 Laptop GPU


In [20]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_username(text):
    url = re.compile(r'@[A-Za-z0-9_]+')
    return url.sub(r'',text)

def pre_process_text(text):
    text = remove_URL(text)
    text = remove_numbers(text)
    text = remove_html(text)
    text = remove_username(text)
    return " ".join(text.split())

In [21]:
df.dropna(subset=['findings'], inplace=True)  # Remove rows with NaN in 'findings'
df = df[df['findings'].str.strip().astype(bool)]  # Remove rows with empty or whitespace 'findings'

df['Problems'] = df['Problems'].str.replace(';', ' ')


In [22]:
print(df['Problems'].value_counts())
print(df['findings'].value_counts())

Problems
normal                                                      1197
No Indexing                                                   87
Lung                                                          79
Calcified Granuloma                                           72
Thoracic Vertebrae                                            59
                                                            ... 
Pulmonary Atelectasis Foreign Bodies Density                   1
Calcified Granuloma Lung Markings                              1
Lung Opacity Markings                                          1
Hernia, Diaphragmatic Bone Diseases, Metabolic Deformity       1
Opacity Granuloma                                              1
Name: count, Length: 1244, dtype: int64
findings
The heart and lungs have XXXX XXXX in the interval. Both lungs are clear and expanded. Heart and mediastinum normal.                                                                                                               

In [23]:
from transformers import T5Tokenizer, AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer


# Splitting source and target into train and eval sets (80-20 split)

# Assuming df is your DataFrame
source = df['Problems'].tolist()
target = df['findings'].tolist()

source_train, source_eval, target_train, target_eval = train_test_split(
    source, target, test_size=0.2, random_state=42
)

# Convert the data into strings
source_train_str = '\n'.join(source_train)
target_train_str = '\n'.join(target_train)
source_eval_str = '\n'.join(source_eval)
target_eval_str = '\n'.join(target_eval)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


# Tokenize the source and target texts
tokenized_source = tokenizer(source_train_str,truncation=True, return_tensors="pt")
tokenized_target = tokenizer(target_train_str, truncation=True, return_tensors="pt")

eval_source = tokenizer(source_eval_str, truncation=True, return_tensors="pt")
eval_target = tokenizer(target_eval_str,truncation=True, return_tensors="pt")

print('Tokenized source has shape', tokenized_source['input_ids'].shape)
print('Tokenized target has shape', tokenized_target['input_ids'].shape)

print('Eval source has shape', eval_source['input_ids'].shape)
print('Eval target has shape', eval_target['input_ids'].shape)


Tokenized source has shape torch.Size([1, 1024])
Tokenized target has shape torch.Size([1, 1024])
Eval source has shape torch.Size([1, 1024])
Eval target has shape torch.Size([1, 1024])


In [24]:
print(tokenized_source)
print(tokenized_target)

{'input_ids': tensor([[11265,   198, 11265,  ...,   198, 18257,  4355]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
{'input_ids': tensor([[  464, 21726,   290,  ...,   914, 15880,    11]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [25]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, source_input_ids, source_attention_mask, target_input_ids, target_attention_mask):
        self.source_input_ids = source_input_ids
        self.source_attention_mask = source_attention_mask
        self.target_input_ids = target_input_ids
        self.target_attention_mask = target_attention_mask

    def __len__(self):
        return len(self.source_input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.source_input_ids[idx],
            'attention_mask': self.source_attention_mask[idx],
            'labels': self.target_input_ids[idx],
            'labels_attention_mask': self.target_attention_mask[idx]
        }

# Assuming you have the tokenized tensors: tokenized_source and tokenized_target
train_dataset = CustomDataset(
    tokenized_source['input_ids'],
    tokenized_source['attention_mask'],
    tokenized_target['input_ids'],
    tokenized_target['attention_mask']
)
eval_dataset = CustomDataset(
    eval_source['input_ids'],
    eval_source['attention_mask'],
    eval_target['input_ids'],
    eval_target['attention_mask']
)


In [26]:
print(train_dataset)
print(eval_dataset)




# Finetuning

In [27]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoModelForSeq2SeqLM

# Initialize the BioClinicalBERT model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use the created dataset here
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Save the trained model to a directory
model.save_pretrained("/FYP_DATASET/results/")


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 14.162193298339844, 'eval_runtime': 0.1884, 'eval_samples_per_second': 5.308, 'eval_steps_per_second': 5.308, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 14.154274940490723, 'eval_runtime': 0.2043, 'eval_samples_per_second': 4.894, 'eval_steps_per_second': 4.894, 'epoch': 2.0}


...

{'eval_loss': 6.410269260406494, 'eval_runtime': 0.2225, 'eval_samples_per_second': 4.495, 'eval_steps_per_second': 4.495, 'epoch': 98.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 6.3911051750183105, 'eval_runtime': 0.2251, 'eval_samples_per_second': 4.443, 'eval_steps_per_second': 4.443, 'epoch': 99.0}
{'loss': 8.764, 'learning_rate': 1e-05, 'epoch': 100.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 6.373225688934326, 'eval_runtime': 0.1854, 'eval_samples_per_second': 5.394, 'eval_steps_per_second': 5.394, 'epoch': 100.0}
{'train_runtime': 70.1488, 'train_samples_per_second': 1.426, 'train_steps_per_second': 1.426, 'train_loss': 8.764032592773438, 'epoch': 100.0}


In [28]:
from transformers import AutoModelForSequenceClassification

loaded_model = AutoModelForSequenceClassification.from_pretrained("/FYP_DATASET/results/")
print(loaded_model)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /FYP_DATASET/results/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [29]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the trained model and tokenizer for BioClinicalBERT
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


# Get user input
user_input = input("Enter your input text: ")

model_input = user_input.replace(';', ' ')

# Tokenize the user input
tokenized_input = tokenizer.encode(model_input, return_tensors="pt", max_length=512, truncation=True)

output = model.generate(tokenized_input, max_length=150, num_return_sequences=1, early_stopping=True)

# Decode the output tokens to text
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)


print("Processed Input:", model_input)
# Display the generated text
print("Generated output:", decoded_output)

c:\Users\Saad Subhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed Input: Mediastinum Aorta, Thoracic Scoliosis
Generated output: Mediastinum Aorta, Thoracic Scoliosis, and the Role of the Biliary System in the Treatment of Gastrointestinal Disorders. Gastroenterology. 2002;116(3):903-912.

[Crossref]

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

Klein, J.

K
